# 1 Импорт библиотек

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

# 2 Загрузка данных

In [20]:
file_path = '../data/tickets.csv'

data = pd.read_csv(file_path, delimiter=';')

In [21]:
# Проверка корректной загрузки данных
print("Пример данных:")
display(data.head())

Пример данных:


,id,title,Type,Description
0,NBI-1061,Smart Task Suggestions for Pipelines,New Feature,Create a feature that suggests tasks for each ...
1,NBI-1092,[Investigate] Error on Large Dataset Imports,Bug,Imports of large datasets sometimes result in ...
2,NBI-1099,Customer invoice attachments missing,Bug,Attachments added to customer invoices are not...
3,NBI-1120,Advanced customer segmentation tools,New Feature,Build a segmentation tool for categorizing cus...
4,NBI-1205,Dynamic Role Templates,New Feature,Create dynamic role templates that administrat...


# 3 Предобработка данных

In [22]:
# Удаляем строки с пропущенными значениями
data = data.dropna(subset=['Description', 'Type'])

In [23]:
# Приведение текста к нижнему регистру
data['Description'] = data['Description'].str.lower()

In [24]:
# Удаление лишних символов (шум)
data['Description'] = data['Description'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [25]:
# Выводим несколько обработанных записей
print("\nОбработанные данные:")
display(data.head())


Обработанные данные:


,id,title,Type,Description
0,NBI-1061,Smart Task Suggestions for Pipelines,New Feature,create a feature that suggests tasks for each ...
1,NBI-1092,[Investigate] Error on Large Dataset Imports,Bug,imports of large datasets sometimes result in ...
2,NBI-1099,Customer invoice attachments missing,Bug,attachments added to customer invoices are not...
3,NBI-1120,Advanced customer segmentation tools,New Feature,build a segmentation tool for categorizing cus...
4,NBI-1205,Dynamic Role Templates,New Feature,create dynamic role templates that administrat...


# 3 Разделение на признаки и целевые переменные, векторизация данных TF-IDF

In [26]:
X = data['Description']  # Признаки
y = data['Type']         # Целевая переменная

## Создание обучающей и валидационной выборки

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nКоличество записей в обучающей выборке: {len(X_train)}")
print(f"Количество записей в валидационной выборке: {len(X_test)}")


Количество записей в обучающей выборке: 1228
Количество записей в валидационной выборке: 308


In [28]:
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("\nРазмерность векторизованных данных:")
print(f"Обучающие данные: {X_train_tfidf.shape}")
print(f"Валидационные данные: {X_test_tfidf.shape}")


Размерность векторизованных данных:
Обучающие данные: (1228, 3106)
Валидационные данные: (308, 3106)


# 4 Обучение модели с использованием метода логистической регрессии

In [29]:
model = LogisticRegression()

model.fit(X_train_tfidf, y_train)

LogisticRegression()

# 5 Проверка работоспособности и оценка модели

In [30]:
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)

print("\nAccuracy модели:", accuracy)


Accuracy модели: 0.8084415584415584


# 6 Сохранение модели

In [31]:
model_path = '../models/ticket_classifier.pkl'

with open(model_path, 'wb') as model_file:
    pickle.dump(model, model_file)